In [1]:
import re
import pandas as pd

beyonce_df = pd.read_csv('/Users/mertakcay/Projects/Knowledge Graphs/KnowledgeGraphBuilder/context.csv')
context_list = beyonce_df['context']#.apply(lambda x : re.sub(r'[^\w\s]', '', x))


In [2]:
sentence_list = []
for context in context_list:
    for sentence in context.split('.'):
        sentence_list.append(sentence)
        
adjectives = ['it','who','which','where','he','she']

In [3]:
prompt_sentences = []
for idx in range(len(sentence_list)):
    if idx != 0: 
        for adjective in adjectives:
            if adjective in sentence_list[idx]:
                prompt_sentences.append('. '.join(sentence_list[idx-1:idx+2]))
                break
    else:
        prompt_sentences.append(sentence)
        

In [4]:
from gpt4all import GPT4All
model = GPT4All("/Users/mertakcay/Projects/Knowledge Graphs/KnowledgeGraphBuilder/mistral-7b-instruct-v0.1.Q4_0.gguf")

In [ ]:
SYS_PROMPT = (
    "You are a ontologist who extracts terms and their relations from a given context. "
    "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
    "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
        "\tTerms may include object, entity, location, organization, person, event, so on.. \n"
        "\tcondition, acronym, documents, service, concept, etc.\n"
        "\tTerms should be as atomistic as possible\n\n"
    "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
        "\tTerms that are mentioned in the same,previous or next sentence..\n"
        "\tTerms can be related to many other terms\n\n"
    "Thought 3: Find out the relation between each such related pair of terms. \n\n"
    "Format your output as a list of json. Each element of the list contains a pair of terms"
    "and the relation between them, like the follwing: \n"
    "[\n"
    "   {\n"
    '       "node_1": "A concept from extracted ontology",\n'
    '       "node_2": "A related concept from extracted ontology",\n'
    '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
    "   }, {...}\n"
    "]"
)

In [5]:
example_term = """
Beyoncé, Destiny's Child
"""
example_groups = """
Person: Beyoncé, Marc Nelson
Group: Destiny's Child, Boyz II Men
"""
example_relation ="""
part_of (Person, Group): Marc Nelson part_of Boyz II Men
"""

In [7]:
import ast
output_lists = list()

for context in prompt_sentences:
    # prompt_ner = f'''
    # <s>[INST] Extract <Subject, Predicate, Object> from {context} in json format [/INST]
    # '''
    prompt_ner = f"""You are a ontologist who extracts terms and their categories from a given context.\n
        You are provided with a context chunk Your task is to extract the ontology of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n
        While traversing through each sentence, Think about the key terms mentioned in it.\n
            \tTerms may include object, entity, location, organization, person, event, so on.. \n
            \tcondition, acronym, documents, service, concept, etc.\n
            \tTerms should be as atomistic as possible\n\n
        Format your output as a list. Each element of the list contains a term and the relation between them, like the follwing: \n
        [term1, term2] \n\n
        Given context: {context}"""
        
    output = model.generate(prompt_ner, max_tokens=8192)
    try:
        first_idx = output.index('[')
        last_idx = output.rindex(']')
    except:
        print(output)
    # output_list = ast.literal_eval(output[first_idx:last_idx+1])
    # print(output_list, type(output_list))
    print(output[first_idx:last_idx+1])
    output_lists.append(output[first_idx:last_idx+1])
    print("----")

["employee", "ABC Company", "Marketing department"]
----
.

        After Destiny's Child disbanded in 2006, Beyoncé began a successful solo career that has earned her numerous awards and accolades, including 28 Grammy Awards. She is widely regarded as one of the greatest music artists of all time.  Her work often explores themes such as love, relationships, identity, feminism, and social justice.

        Beyoncé has been married to rapper Jay-Z since April 4, 2008. They have three children together: Blue Ivy Carter (born January 7, 2013), Sir Carter (born February 6, 2017), and Rumi Carter (born July 23, 2019).

        Beyoncé is also known for her philanthropic work. She founded the BeyGOOD Initiative in 2010 to support various charitable causes, including education, disaster relief, and military families. In addition, she has partnered with numerous organizations to promote social justice issues such as Black Lives Matter and LGBTQ+ rights.

        Beyoncé's personal life has bee

In [36]:
full_output = ''
for sample in output_lists:
    full_output = full_output + sample.replace('\n',' ').replace('[',',').replace(']',',').replace('"','')

full_output = ' '.join(full_output.split())
full_output = full_output.replace(',,',',')
term_list = full_output.split(',')
term_list = list(set(term_list))
term_list

['',
 '000 copies in its first week',
 'Knowles family',
 ' Album',
 'singing-songwriter',
 ' international single',
 ' 2011 Glastonbury Festival',
 'Song',
 ' monogamy',
 'Satellite Award nomination',
 ' separation',
 'Rolling Stone',
 " Beyoncé's parents",
 ' solo albums',
 ' number-one',
 'm. Her debut album Dangerously in Love sold over 11 million copies worldwide',
 ' signed management deal with group',
 ' Duet',
 ' debut',
 'certified gold by the Recording Industry Association of America (RIAA) for shipping over 500',
 '2009 MTV Video Music Awards',
 ' top-five',
 ' I Care',
 'Billboard 200',
 " continued performing as an opening act for other established R&B girl groups such as TLC and Destiny's Child",
 'note 2',
 ' Artist of the Millennium',
 ' Hummed',
 ' Super Bowl',
 ' music',
 ' acting',
 ' Grammy Award',
 ' female artist',
 'January 2010',
 ' Egyptian pyramids',
 ' Beyoncé',
 ' philanthropic work',
 ' depression',
 'Déjà Vu',
 "St. John's United Methodist Church",
 ' Unit

In [44]:
term_list[0:50]

['',
 '000 copies in its first week',
 'Knowles family',
 ' Album',
 'singing-songwriter',
 ' international single',
 ' 2011 Glastonbury Festival',
 'Song',
 ' monogamy',
 'Satellite Award nomination',
 ' separation',
 'Rolling Stone',
 " Beyoncé's parents",
 ' solo albums',
 ' number-one',
 'm. Her debut album Dangerously in Love sold over 11 million copies worldwide',
 ' signed management deal with group',
 ' Duet',
 ' debut',
 'certified gold by the Recording Industry Association of America (RIAA) for shipping over 500',
 '2009 MTV Video Music Awards',
 ' top-five',
 ' I Care',
 'Billboard 200',
 " continued performing as an opening act for other established R&B girl groups such as TLC and Destiny's Child",
 'note 2',
 ' Artist of the Millennium',
 ' Hummed',
 ' Super Bowl',
 ' music',
 ' acting',
 ' Grammy Award',
 ' female artist',
 'January 2010',
 ' Egyptian pyramids',
 ' Beyoncé',
 ' philanthropic work',
 ' depression',
 'Déjà Vu',
 "St. John's United Methodist Church",
 ' Unit

In [50]:
term_groups = list()
window_size = 50
for context_window in range(window_size,len(term_list),window_size):
    context = ', '.join(term_list[context_window - window_size:context_window])
    
    prompt_groupping = """Given a list of terms, group them together based on how similar their meanings are. Terms within a group should share a common theme or concept.\n
    Term List: %s \n
    Output should be json format""" % context
    output = model.generate(prompt_groupping, max_tokens=8192)
    print(output)
    term_groups.append(output)

:
    
    {
        "Singer": ["Beyoncé", "singing-songwriter"],
        "Albums and Songs": ["Dangerously in Love", "I Care", "Déjà Vu", "Rolling Stone", "Super Bowl", "Hummed", "Performing", "Dreamgirls", "Song", "DVD release"],
        "Philanthropy and Awards": ["NAACP Image Award nomination", "Grammy Award", "Artist of the Millennium", "m. Her debut album Dangerously in Love sold over 11 million copies worldwide,  signed management deal with group,  continued performing as an opening act for other established R&B girl groups such as TLC and Destiny's Child"],
        "Family": ["Knowles family", "Beyoncé's parents"],
        "Acting": ["acting"],
        "Depression": ["depression"],
        "International Recognition": ["international single", "2011 Glastonbury Festival", "New Zealand"]
    }
.
:  { "Concert Tours": ["concert tour", "stadium tour"], "Celebrities and Music Industry": ["Foxxy Cleopatra", "Jay Z", "Beyoncé", "Dwayne Wiggins", "Obsessed", "WikiLeaks", "New Orleans",

In [7]:
import ast
output_lists = list()

for context in prompt_sentences:

    prompt_ner = """You are a network graph maker who extracts terms and their relations from a given context. 
        You are provided with a context chunk (delimited by ```) Your task is to extract the ontology 
        of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n
        Thought 1: While traversing through each sentence, Extract about the key terms mentioned in it.\n
            \tTerms may include object, entity, location, organization, person, \n
            \tcondition, acronym, documents, service, concept, etc.\n
            \tTerms should be as atomistic as possible\n\n
        Thought 2: Given a list of terms, group them together based on how similar their meanings are. Terms within a group should share a common theme or concept.\n
            \tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n
            \tTerms can be related to many other terms\n\n
        Thought 3: Find out the relation between each such related pair of terms. \n\n
        Context: %s \n\n
        
        Format your output as a list of json. Each element of the list contains a pair of terms and the relation between them, like the follwing: \n
        [\n
           {\n
               "node_1": "A concept from extracted ontology",\n
               "node_2": "A related concept from extracted ontology",\n
               "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n
           }, {...}\n
        ] \n
        
        """ % context
        
    output = model.generate(prompt_ner, max_tokens=8192)
    
    print(output)
    print("----")
    break


        For example: 

        [

            {

                "node_1": "AI",

                "node_2": "Machine Learning",

                "edge": "Both are concepts from extracted ontology, AI is mentioned in the first sentence and Machine Learning is mentioned in the second sentence"

            }, {...}

        ]
----


In [ ]:
import ast
output_lists = list()

for context in prompt_sentences:
    # prompt_ner = f'''
    # <s>[INST] Extract <Subject, Predicate, Object> from {context} in json format [/INST]
    # '''
    prompt_ner = f'''
    <s>[INST] You are a ontologist who extracts terms and their categories from a given context. Your task is to generate a valid JSON object based on the given information. So for instance the following: \n
        {context}[/INST]
        [Beyoncé, Marc Nelson, term1, term2, ...]
        </s>
    '''
        
    output = model.generate(prompt_ner, max_tokens=8192)
    try:
        first_idx = output.index('[')
        last_idx = output.rindex(']')
    except:
        print(output)
    # output_list = ast.literal_eval(output[first_idx:last_idx+1])
    # print(output_list, type(output_list))
    print(output[first_idx:last_idx+1])
    output_lists.append(output[first_idx:last_idx+1])
    print("----")

In [ ]:
    # prompt_ner = """You are a ontologist who extracts terms and their categories from a given context.\n
    #     Task 1: Your task is to extract the ontology of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n
    #     While traversing through each sentence, Think about the key terms mentioned in it.\n
    #         \tTerms may include object, entity, location, organization, person, event, so on.. \n
    #         \tcondition, acronym, documents, service, concept, etc.\n
    #         \tTerms should be as atomistic as possible\n\n
    #     Task 2: Your task is to group each term in terms\n\
    #         \tTerms is given from Task 1\n\n
    #     Task 3: Find relation between each related pair from given context\n
    #         \tUse terms from Task 1 and Task 2 to find relations between related pair\n\n
    #     Format your output as a list of json. Each element of the list contains a pair of terms and the relation between them, like the follwing: \n
    #     [\n
    #         {\n
    #          'node_1':'Term extracted by ontology',\n
    #          'node_2':'Term extracted by ontology and related to node_1',\n
    #          'relation':'relationship between the two concepts, node_1 and node_2 in one or two sentences'\n 
    #         }, {...}, {...}, {...}, {...}, {...}
    #     ]
    #     Given context: %s""" % context